In [8]:
# -*- coding:utf-8 -*-
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import time, os
import api_key as api
import common_tools as common_tools

api_key = api.openai_api_key()
# 将API Key保存为环境变量
os.environ["OPENAI_API_KEY"] = api_key

base_url= 'http://192.168.10.107:11434/v1'
model_name='qwen2:72b'
# model_name='qwen:110b'

model = ChatOpenAI(
    base_url=base_url,
    model_name=model_name,
    temperature=1.1,
    streaming=True
)

output_parser = StrOutputParser()

def translate_once(prompt, origin_content):
    chain = prompt | model | output_parser
    response = chain.invoke({"input": origin_content})
    out_content = common_tools.extract_translation(response)
    out_content = common_tools.modify_text(out_content)
    result = out_content
    # 使用format()方法拼接字符串并加入换行符
    # result = "{}\n\n{}".format(origin_content, out_content)
    return result

# 步骤二：处理内容
def process_chunks(prompt, chunks):
    processed_chunks = []
    for chunk in chunks:
        modified_chunk = translate_once(prompt, chunk)
        processed_chunks.append(modified_chunk)
    return processed_chunks

def merge_and_save_chunks(chunks, filename):
    merged_text = '\n\n'.join(chunks)
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(merged_text)

def translate():
    prompt_content = common_tools.read_file('./translate_prompt.md')
    origin_content = common_tools.read_file('/Users/Daglas/Desktop/input.md')
    prompt = ChatPromptTemplate.from_messages([
        ("system", prompt_content),
        ("user", "{input}")
    ])
    chunks = common_tools.split_text_by_length(origin_content, 2000)
    processed_chunks = process_chunks(prompt, chunks)
    merge_and_save_chunks(processed_chunks, '/Users/Daglas/Desktop/output.md')

start_time = time.time()
print('waiting...\n')
translate()
end_time = time.time()
print('Time Used: ' + str((end_time - start_time)/60) + 'min')

waiting...

Time Used: 4.342482634385427min
